# Streaming with Kafka and Spark

Here I try to implement a basic pipeline for the project conecting kafka with spark.

### Computer setting
I downloaded and located in my home the spark file **spark-3.1.2-bin-hadoop3.2** and also the kafka file **kafka_2.13-2.7.0**

## Get Kafka and Spark ready

### Standalone cluster deployment

We can now initialize all the required variables with `findspark.init()` by passing the path to the spark folder we downloaded previously.

In [1]:
import findspark
findspark.init('/usr/local/spark/')

First we need to start the master, This will spin up the spark master with address spark://localhost:7077 and a cluster dashboark at localhost:8080.

We can now create a worker

In [2]:
%%script bash --no-raise-error

# start master 
$SPARK_HOME/sbin/start-master.sh --host localhost \
    --port 7077 --webui-port 8080
    
# start worker
$SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
    --cores 2 --memory 1g



starting org.apache.spark.deploy.master.Master, logging to /usr/local/spark//logs/spark-saverio-org.apache.spark.deploy.master.Master-1-saverio-PU301LA.out
starting org.apache.spark.deploy.worker.Worker, logging to /usr/local/spark//logs/spark-saverio-org.apache.spark.deploy.worker.Worker-1-saverio-PU301LA.out


## Create the spark session

We can now create the spark session. With the following command we are asking to the master (and resource manager) to create an application with required resources and configurations. In this case we are using all the default options.

In [3]:
from pyspark.sql import SparkSession

KAFKA_BOOTSTRAP_SERVERS = ''

    
spark = SparkSession.builder \
    .master("spark://localhost:7077")\
    .appName("Spark Streaming")\
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")\
    .getOrCreate()
  


21/08/30 22:53:13 WARN Utils: Your hostname, saverio-PU301LA resolves to a loopback address: 127.0.1.1; using 192.168.1.24 instead (on interface wlp3s0)
21/08/30 22:53:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/saverio/.ivy2/cache
The jars for the packages stored in: /home/saverio/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a8508cb9-806d-4149-8abb-58fdd72f6bee;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.1 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 628ms :: artifacts dl 15ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from centr

In [4]:
spark

## KAFKA

In [5]:
KAFKA_HOME = '/usr/local/kafka'
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

In [8]:
#By some reason I can't launch this from here using OS, so i open the terminals in the KAFKA_HOME folder
# and launch the zookeper and the kafka server comands manually


# Start Zookeeper
# bin/zookeeper-server-start.sh config/zookeeper.properties 
#os.system('{0}/bin/zookeeper-server-start.sh {0}/config/zookeeper.properties'.format(KAFKA_HOME)) 
    
# Start one Kafka Broker
#bin/kafka-server-start.sh config/server.properties
#os.system('{0}/bin/kafka-server-start.sh {0}/config/server.properties'.format(KAFKA_HOME)) 

### Create the topics for kafka

In [6]:
from kafka.admin import KafkaAdminClient, NewTopic
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    )

#Here we will inject the data
new_topic_a = NewTopic(name='Experiment_measurements', 
                       num_partitions=1, 
                       replication_factor=1)

#Here we inject the number of processed hits, post cleaning
new_topic_b = NewTopic(name='results', 
                       num_partitions=1, 
                       replication_factor=1)

kafka_admin.create_topics(new_topics=[new_topic_a,new_topic_b])


CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='Experiment_measurements', error_code=0, error_message=None), (topic='results', error_code=0, error_message=None)])

In [7]:
kafka_admin.list_topics()

['results', 'Experiment_measurements']

## KAFKA - SPARK INTEGRATION

### Read the data from the kafka topic (define the consumer)

In [8]:
inputDF = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
    .option('subscribe', 'Experiment_measurements')\
    .load()

In [9]:
#inputDF.printSchema()

In [10]:
from pyspark.sql.functions import from_json, col, when
from pyspark.sql.types import StructField, StructType, DoubleType, IntegerType

## the schema of the json data format used to create the messages
schema = StructType(
        [
                StructField("HEAD",        IntegerType()),
                StructField("FPGA",        IntegerType()),
                StructField("TDC_CHANNEL", IntegerType()),
                StructField("ORBIT_CNT",   DoubleType()),
                StructField("BX_COUNTER",  IntegerType()),
                StructField("TDC_MEAS",    DoubleType())
        ]  
)

## a new DF can be created from the previous by using the pyspark.sql functions
jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

In [11]:
jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: integer (nullable = true)
 |    |-- FPGA: integer (nullable = true)
 |    |-- TDC_CHANNEL: integer (nullable = true)
 |    |-- ORBIT_CNT: double (nullable = true)
 |    |-- BX_COUNTER: integer (nullable = true)
 |    |-- TDC_MEAS: double (nullable = true)



In [12]:
#jsonDF.writeStream\
#   .outputMode("append")\
#   .format("console")\
#   .start()\
#   .awaitTermination()

In [13]:
flatDF = jsonDF.selectExpr("value.HEAD", 
                           "value.FPGA", 
                           "value.TDC_CHANNEL",
                           "value.ORBIT_CNT",
                           "value.BX_COUNTER",
                           "value.TDC_MEAS")

In [14]:
flatDF.printSchema()

root
 |-- HEAD: integer (nullable = true)
 |-- FPGA: integer (nullable = true)
 |-- TDC_CHANNEL: integer (nullable = true)
 |-- ORBIT_CNT: double (nullable = true)
 |-- BX_COUNTER: integer (nullable = true)
 |-- TDC_MEAS: double (nullable = true)



In [15]:
#flatDF.writeStream\
#   .outputMode("append")\
#   .format("console")\
#   .start()\
#   .awaitTermination()

### SPARK processing

In [16]:
import json
import numpy as np
import time

#Keep the events where "HEAD"=2
cleanDF = flatDF.where(col('HEAD')==2)  

In [17]:
def computations(DF, epoch):
    #This function perform the whole operations on the received batch,
    #

    #As the 4 calculations that we have to perform are done foe each chamber we set 4 dataframes
    chamber_1 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_2 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))
    chamber_3 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_4 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))

    #Now we can count the number of events in each chamber
    n_c1 = chamber_1.count()
    n_c2 = chamber_2.count()
    n_c3 = chamber_3.count()
    n_c4 = chamber_4.count()

    #Total number of events
    n = n_c1 + n_c2 + n_c3 + n_c4


    #Histograms    
    h_c1 = chamber_1.groupBy('TDC_CHANNEL').count().collect()
    h_c2 = chamber_2.groupBy('TDC_CHANNEL').count().collect()
    h_c3 = chamber_3.groupBy('TDC_CHANNEL').count().collect()
    h_c4 = chamber_4.groupBy('TDC_CHANNEL').count().collect()

    h_active_1 = chamber_1.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_2 = chamber_2.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_3 = chamber_3.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    h_active_4 = chamber_4.groupBy('TDC_CHANNEL','ORBIT_CNT').count().collect()
    
    
    #Organise the results to send them to one topic as a dictionary
    results = {'Total_events': n,
              'Events_per_chamber': [n_c1,n_c2,n_c3,n_c4],
              'Histogram_1': [h_c1, h_c2, h_c3, h_c4],
              'Histogram_2': [h_active_1,h_active_2,h_active_3,h_active_4]}
    
    #publish the results in the "results" topic for further usage
    producer.send(topic='results', value=json.dumps(results).encode('utf-8'))
    producer.flush()

In [18]:
def computations_2(DF, epoch):
    start=time.time()
    #This function perform the whole operations on the received batch,
    #

    #As the 4 calculations that we have to perform are done foe each chamber we set 4 dataframes
    chamber_1 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_2 = DF.filter((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))
    chamber_3 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=0) & (col("TDC_CHANNEL")<=63))
    chamber_4 = DF.filter((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64) & (col("TDC_CHANNEL")<=127))

    #Initialize results dictionary
    results = {}
    results["Total Count"] = {}
    chamber_name = ["Chamber_1", "Chamber_2", "Chamber_3", "Chamber_4"]
    for chamber in chamber_name:
        results[chamber] = {}
        results[chamber]["Count"] = {}
        
    #Now we can count the number of events in each chamber
    results["Chamber_1"]["Count"] = chamber_1.count()
    results["Chamber_2"]["Count"] = chamber_2.count()
    results["Chamber_3"]["Count"] = chamber_3.count()
    results["Chamber_4"]["Count"] = chamber_4.count()

    #Total number of events
    results["Total Count"] = results["Chamber_1"]["Count"] + results["Chamber_2"]["Count"] + \
                             results["Chamber_3"]["Count"] + results["Chamber_4"]["Count"]
    

    # Compute histograms for each chamber   
    i=0    
    for chamber in [chamber_1, chamber_2, chamber_3, chamber_4]:
        #Initialize dictionary partitions to save the results
        for hist in ["Hist_1","Hist_2"]:
            results[chamber_name[i]][hist] = {}
            results[chamber_name[i]][hist]["Bins"] = {}
            results[chamber_name[i]][hist]["Counts"] = {}
        
        if(chamber.count()!=0): 
            #Histogram 1
            bins, counts = (
                chamber.select("TDC_CHANNEL")
                .rdd.map(lambda x: x.TDC_CHANNEL)
                .histogram(list(np.arange(0,170,5)))
            )
            
            results[chamber_name[i]]["Hist_1"]["Bins"] = bins
            results[chamber_name[i]]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            bins, counts = (
            chamber.groupBy("TDC_CHANNEL","ORBIT_CNT")
            .count()
            .select("ORBIT_CNT")
            .rdd.map(lambda x: x.ORBIT_CNT)
            .histogram(list(np.arange(6.e5,1.e7,0.5e6)))
            )
            
            results[chamber_name[i]]["Hist_2"]["Bins"] = bins
            results[chamber_name[i]]["Hist_2"]["Counts"] = counts            
            
        else:
            #Histogram 1
            results[chamber_name[i]]["Hist_1"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[chamber_name[i]]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            results[chamber_name[i]]["Hist_2"]["Bins"] = list(np.arange(6.e5,1.e7,0.5e6))
            counts = list(np.arange(6.e5,1.e7,0.5e6)* 0) 
            results[chamber_name[i]]["Hist_2"]["Counts"] = counts
        i +=1
    
    end =time.time()
    print("Time =",end-start)
    producer.send(topic="results", value= str(results).encode('utf-8'))
    producer.flush()

In [19]:
def computations_3(DF, epoch):
    start=time.time()
    #This function perform the whole operations on the received batch,
    
    #Add a column with the chamber number
    DF_new = DF.withColumn('chamber',when((col("FPGA") == 0) & (col("TDC_CHANNEL")<=63),1).
                                 when((col("FPGA") == 0) & (col("TDC_CHANNEL")>=64),2).
                                 when((col("FPGA") == 1) & (col("TDC_CHANNEL")<=63),3).
                                 when((col("FPGA") == 1) & (col("TDC_CHANNEL")>=64),4)).\
                                 select([ col('TDC_CHANNEL'), col('ORBIT_CNT'),
                                    col('BX_COUNTER'),col('TDC_MEAS'),
                                    col('chamber')])
    DF_new.persist()
    #DF_new.show()
    #Initialize results dictionary
    results = {}
    results["Total Count"] = {}
    results["Index"] = time.time()
    chamber_name = ["Chamber_1", "Chamber_2", "Chamber_3", "Chamber_4"]
    for chamber in chamber_name:
        results[chamber] = {}
        results[chamber]["Count"] = {}
        for hist in ["Hist_1","Hist_2"]:
            results[chamber][hist] = {}
            results[chamber][hist]["Bins"] = {}
            results[chamber][hist]["Counts"] = {}
        
    # Compute histograms for each chamber   
    for i in [1,2,3,4]:
        #Now we can count the number of events in each chamber
        chamber = DF_new.filter(col("chamber") == i).persist()
        results[f"Chamber_{i}"]["Count"] = chamber.count()
        
        if(results[f"Chamber_{i}"]["Count"]!=0):
            
            #Histogram 1
            bins, counts = (
            chamber.select("TDC_CHANNEL")
                 .rdd.map(lambda x: x.TDC_CHANNEL)
                 .histogram(list(np.arange(0,170,5)))
            )
            
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = bins
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts
            
            #Histogram 2
            bins, counts = (
            chamber.groupBy("TDC_CHANNEL","ORBIT_CNT")
            .count()
            .select("ORBIT_CNT")
            .rdd.map(lambda x: x.ORBIT_CNT)
            .histogram(list(np.arange(6.e5,1.e7,0.5e6)))
            )
            
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = bins
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts            
                
        else:
            #Histogram 1
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts
            
             #Histogram 2
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = list(np.arange(6.e5,1.e7,0.5e6))
            counts = list(np.arange(6.e5,1.e7,0.5e6)* 0) 
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts
        chamber = DF_new.filter(col("chamber") == i).unpersist()  
        
    results["Total Count"] = results["Chamber_1"]["Count"] + results["Chamber_2"]["Count"] + \
                             results["Chamber_3"]["Count"] + results["Chamber_4"]["Count"]
    end =time.time()
    print("Time =",end-start)
       
    
    producer.send(topic="results", value= str(results).encode('utf-8'))
    producer.flush()

In [20]:
def computations_7(DF, epoch):
    start=time.time()
    #This function perform the whole operations on the received batch,
    
    #Add a column with the chamber number
    #DF_clean     = DF.filter(col("HEAD")==2)
    DF_hit = DF.withColumn('CHAMBER',when(col("FPGA") == 0, 
                                                when(col("TDC_CHANNEL")<=63,1).\
                                                otherwise(2)).\
                                           otherwise(when(col("TDC_CHANNEL")<=63,3).\
                                                     otherwise(when(col("TDC_CHANNEL")<128,4))
                                           )).\
                                           select([ col('TDC_CHANNEL'), col('ORBIT_CNT'),
                                           col('BX_COUNTER'),col('TDC_MEAS'),
                                           col('CHAMBER')])
    
    #Initialize results dictionary
    results = {}
    results["Total Count"] = {}
    results["Index"] = time.time()
    chamber_name = ["Chamber_1", "Chamber_2", "Chamber_3", "Chamber_4"]
    for chamber in chamber_name:
        results[chamber] = {}
        results[chamber]["Count"] = {}
        for hist in ["Hist_1","Hist_2","Hist_3","Hist_4"]:
            results[chamber][hist] = {}
            results[chamber][hist]["Bins"] = {}
            results[chamber][hist]["Counts"] = {}
    
    
   
    #We prepare the scilantor data
    #First we filter the events encoding the passage time,
    #then we add the PASSAGE time for each event 
    #Finally if we have two scilantor hits within the same orbit we keep 
    #the one with the smallertime
    DF_scintillator = DF_hit.filter((col("CHAMBER").isNull())).\
                          withColumn("PASSAGETIME", 25 * (col("ORBIT_CNT") * 3564 + col("BX_COUNTER") + col("TDC_MEAS")/30)).\
                          drop("TDC_CHANNEL").drop("BX_COUNTER").\
                          drop("TDC_MEAS").drop("CHAMBER").\
                          groupBy("ORBIT_CNT").min("PASSAGETIME").\
                          withColumnRenamed("ORBIT_CNT","ORBIT_CNT_sci").\
                          withColumnRenamed("min(PASSAGETIME)","PASSAGETIME")
    
    #Drop the columns with null values from DF_hit
    DF_hit.na.drop(subset=["CHAMBER"])
        
    # scintillator time offset by Chamber
    time_offset_by_chamber = {
    1: 95.0 - 1.1, # Ch 0
    2: 95.0 + 6.4, # Ch 1
    3: 95.0 + 0.5, # Ch 2
    4: 95.0 - 2.6, # Ch 3
    }

    # Compute histograms for each chamber   
    for i in [1,2,3,4]:     
        #Now we can count the number of events in each chamber
        chamber = DF_hit.filter(col("CHAMBER") == i).persist()
        results[f"Chamber_{i}"]["Count"] = chamber.count()
        
        if(results[f"Chamber_{i}"]["Count"]!=0):
            
            #Histogram 1
            bins_1, counts_1 = (
            chamber.select("TDC_CHANNEL")
                 .rdd.map(lambda x: x.TDC_CHANNEL)
                 .histogram(list(np.arange(0,170,5)))
            )
            
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = bins_1
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts_1
            
            #Histogram 2
            bins_2, counts_2 = (
            chamber.groupBy("TDC_CHANNEL","ORBIT_CNT")
            .count()
            .select("ORBIT_CNT")
            .rdd.map(lambda x: x.ORBIT_CNT)
            .histogram(list(np.arange(6.e5,1.e7,0.5e6)))
            )
            
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = bins_2
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts_2            
            
            
            #keep only the hits with a scintillator signal within the same orbit
            chamber_sci = chamber.join(DF_scintillator,chamber.ORBIT_CNT ==  DF_scintillator.ORBIT_CNT_sci,"inner")
#Add the ABSSOLUTETIME 
            chamber_sci = chamber_sci.withColumn("ABSOLUTETIME",
                             25 * (col("ORBIT_CNT") * 3564 + col("BX_COUNTER") + col("TDC_MEAS")/30))
            #Drop useless data
#            chamber_sci = chamber_sci.drop("HEAD").drop("FPGA").drop("BX_COUNTER").drop("ORBIT_CNT_sci")
  
            #Add DRIFTIME
            chamber_sci = chamber_sci.withColumn("DRIFTIME",col("ABSOLUTETIME")-col("PASSAGETIME") + time_offset_by_chamber[i])#.show()

        
            #Histogram 3
            bins_3, counts_3 = (
            chamber_sci.select("TDC_CHANNEL")
                 .rdd.map(lambda x: x.TDC_CHANNEL)
                 .histogram(list(np.arange(0,170,5)))
            )    
            results[f"Chamber_{i}"]["Hist_3"]["Bins"] = bins_3
            results[f"Chamber_{i}"]["Hist_3"]["Counts"] = counts_3
            
            #Histogram 4
            bins_4, counts_4 = (
            chamber_sci.select("DRIFTIME")
                 .rdd.map(lambda x: x.DRIFTIME)
                 .histogram(list(np.arange(0,1000,10)))
            )
            results[f"Chamber_{i}"]["Hist_4"]["Bins"] = bins_4
            results[f"Chamber_{i}"]["Hist_4"]["Counts"] = counts_4
            
            
            
        else:
            #Histogram 1
            results[f"Chamber_{i}"]["Hist_1"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[f"Chamber_{i}"]["Hist_1"]["Counts"] = counts
            
             #Histogram 2
            results[f"Chamber_{i}"]["Hist_2"]["Bins"] = list(np.arange(6.e5,1.e7,0.5e6))
            counts = list(np.arange(6.e5,1.e7,0.5e6)* 0) 
            results[f"Chamber_{i}"]["Hist_2"]["Counts"] = counts
            
            #Histogram 3
            results[f"Chamber_{i}"]["Hist_3"]["Bins"] = list(np.arange(0,170,5))
            counts = list(np.arange(0,170,5)* 0) 
            results[f"Chamber_{i}"]["Hist_3"]["Counts"] = counts
            
             #Histogram 4
            results[f"Chamber_{i}"]["Hist_4"]["Bins"] = list(np.arange(-100,1000,10))
            counts = list(np.arange(-100,1000,10)* 0) 
            results[f"Chamber_{i}"]["Hist_4"]["Counts"] = counts
        chamber.unpersist()
        
    results["Total Count"] = results["Chamber_1"]["Count"] + results["Chamber_2"]["Count"] + \
                             results["Chamber_3"]["Count"] + results["Chamber_4"]["Count"]
    end =time.time()
    end =time.time()
    print("Time =",end-start)
       
    producer.send(topic="results", value= str(results).encode('utf-8'))
    #producer.flush()

In [21]:
from kafka import KafkaProducer

#Send the results to the kafka topic
#Initialize the producer
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)


In [ ]:
#Trigger the processing
cleanDF.writeStream\
    .foreachBatch(computations_7)\
    .trigger(processingTime='6 second')\
    .start()\
    .awaitTermination()

21/08/30 22:53:47 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e47c420a-b227-4686-8d84-d2322f7b83c9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Time = 4.535778999328613


21/08/30 22:53:55 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 7581 milliseconds
21/08/30 22:55:12 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 72919 milliseconds


Time = 72.60547423362732


21/08/30 22:56:23 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 70863 milliseconds


...

Time = 61.21691346168518


21/08/30 22:59:29 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 57697 milliseconds


Time = 57.57320857048035


21/08/30 23:00:27 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 57234 milliseconds


Time = 57.11191463470459


In [ ]:
spark.stop()

If you also want to delete any data of your local Kafka environment including any events you have created along the way, run the command:

`` $ rm -rf /tmp/kafka-logs /tmp/zookeeper `` 